In [45]:
import statsmodels.api as sm
import pandas as pd
import numpy as np
import math
import scipy.stats
import bisect

In [61]:
#This is the primary function. It takes Y and X just like sm.OLS(Y,X), however I added alpha
#because the user may want to specify the level of confidence for calculating the critical t-value

#I assume that Y and X are np arrays
#alpha is the confidence level. Other softwares use alpha = 0.25
def forward_section(Y, X, alpha):
    
    n = np.shape(X)[0] #number of observations
    k = np.shape(X)[1] #number of regressors
    
    #check if the data includes a constant
    includes_constant = np.array_equiv(X[:,0],np.ones(n))
    
    #initializes the corrolation coefficient matrix
    corr_coeffs = np.corrcoef(X[:, range(includes_constant, k)].T, Y.T)
    
    #keeps track of the remaining regressors
    remaining_regressors = list(range(includes_constant, k))
    
    # a list to store the included regressors
    included_regressors = []
    
    #get the position and value of the largest correlation coefficient
    pos, r = get_max_corr(corr_coeffs)
    
    #calculate the t-statistic
    t = t_stat(r, n)
    
    #calculate the critical value to determine if a regressor should be included
    t_critical = abs(scipy.stats.t.ppf(q=alpha/2,df=n - 1)) 
    
    while len(remaining_regressors) > 0 and t > t_critical:
        
        #there is possibly a miscalculation with t. My generated values were close 
        #to what the textbook had. I am not sure if it is a truncation error OR if 
        #I am not calculating subsequent correlation matrices correctly. 
        #print(t)
        
        #keep the regressors in sorted order, because otherwise the model will incorrectly label
        #the regressors
        bisect.insort(included_regressors, remaining_regressors[pos])
       
        #delete the regressor, so that we can keep track of their index in corr_coeffs
        del remaining_regressors[pos]
        
        #compute the next corr matrix
        corr_coeffs = compute_next_corr_matrix(corr_coeffs, pos)
        
        #get the position and value of the largest correlation coefficient
        pos, r = get_max_corr(corr_coeffs)
        
        #calculate the t-statistic
        t = t_stat(r, n)
        
        #calculate the critical value to determine if a regressor should be included
        t_critical = abs(scipy.stats.t.ppf(q=alpha/2,df=n - len(included_regressors) - 1))
    
    #include the constant in the model if it exists
    if includes_constant == True:
        bisect.insort(included_regressors, 0)
    
    #return a OLS model    
    return sm.OLS(Y, X[:, included_regressors])
    

In [62]:
#calculates the t_stat using the precalculated r for optimization
#Unfortunatly, I do not think this method is accurate after several iterations of calculating
#the corr_coeffs matrix

def t_stat(r, n):
    return abs(r * math.sqrt((n-2) / (1-r*r)))

In [31]:
#We need a special function to get the max correlation, because the row will contain 1 at the 
#end, which will always be the max. There is prob a better way to do this
def get_max_corr(corr_matrix):
    max_corr = 0;
    max_corr_pos = -1;
    for i in range (0, np.shape(corr_matrix)[1] - 1): #get the number of cols, -1 because the last element will be 1 and will be the max 
        if abs(corr_matrix[np.shape(corr_matrix)[1] - 1][i]) > max_corr:
            max_corr = abs(corr_matrix[np.shape(corr_matrix)[1] - 1][i])
            max_corr_pos = i
    return max_corr_pos, max_corr

In [63]:
#since there is a truncation error or a calculation error, a formal proof is likely needed
# to show that this function is indeed calculating the correct r values

# k is the correlation matrix, x is the position of the regressor that was 
#already added to the model

def compute_next_corr_matrix(corr, x): 
    
    #a) delete the column with the given element
    corr = np.delete(corr, x, axis=1)
    
    #b) save the row with the given element
    g = np.array(corr[x])
    g = g.reshape(np.shape(corr)[1], 1)
    
    #c) delete the row with the given element
    corr = np.delete(corr, x, axis=0)
    
    #d) Matrix multiply the row from part b
    g2 = g @ g.T #4 X 4 matrix now
    
    #e) subtract the matrices
    numerator = corr - g2
    
    #f)calculate the b^2 so it is a 
    denom = np.sqrt((1 - np.square(g)) @ (1 - np.square(g)).T)
    return numerator / denom

In [64]:
df  = pd.read_excel('Test_Data.xlsx', sheet_name='Test_data')
df_n = df.to_numpy()
X = df_n[:, 1:(df_n.shape[1])]
Y = df_n[:, [0]]
#add a column of 1's to the front to account for the constant
X = sm.add_constant(X)


In [65]:
my_model = forward_section(Y, X, 0.25)
res = my_model.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.982
Model:                            OLS   Adj. R-squared:                  0.976
Method:                 Least Squares   F-statistic:                     166.8
Date:                Thu, 04 Nov 2021   Prob (F-statistic):           3.32e-08
Time:                        14:48:01   Log-Likelihood:                -26.933
No. Observations:                  13   AIC:                             61.87
Df Residuals:                       9   BIC:                             64.13
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         71.6483     14.142      5.066      0.0

In [118]:
#this function returns the partial correlation of x2 to y, given that x1 is included in the model
#we dont need y, becase we are assuming that is what we want to find the correlation for
#x1 should be the row / col number of the regresssor that is already included in the model.
#x2 is the regressor we need to find out the partial correlation for
#cm is the correlation matrix
#function jsut for basic testing
def partial_r(y, x1, x2, cm):
    return (cm[y][x2] - cm[y][x1]*cm[x1][x2]) / (math.sqrt((1-cm[y][x1]*cm[y][x1])*(1-cm[x1][x2]*cm[x1][x2])))